In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# fad_embed

> Generate embeddings from audio files

In [ ]:
#| default_exp fad_embed

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import argparse
import laion_clap 
from laion_clap.training.data import get_audio_features
from accelerate import Accelerator
import warnings
import torch

from aeiou.core import get_device, load_audio, get_audio_filenames, makedir
from aeiou.datasets import AudioDataset
from aeiou.hpc import HostPrinter
from torch.utils.data import DataLoader
from pathlib import Path

In [ ]:
#|export 

def setup_embedder(model_choice, device):
    "load the embedder model"
    embedder = None
    if model_choice == 'clap':
        clap_fusion, clap_amodel = True, "HTSAT-base"
        #doesn't work:  warnings.filterwarnings('ignore')  # temporarily disable CLAP warnings as they are super annoying. 
        clap_module = laion_clap.CLAP_Module(enable_fusion=clap_fusion, device=device, amodel=clap_amodel).requires_grad_(False).eval()
        clap_ckpt_path = os.environ['CLAP_CKPT']  # you'll need access to this .ckpt file
        if clap_ckpt_path:
            #print(f"Loading CLAP from {clap_ckpt_path}")
            clap_module.load_ckpt(ckpt=clap_ckpt_path, verbose=False)
        else:
            clap_module.load_ckpt(model_id=1, verbose=False)
        #warnings.filterwarnings("default")   # turn warnings back on. 
        embedder = clap_module # synonyms 
    else:
        raise ValueError("Sorry, other models not supported yet")
        
    return embedder



def embed_all(args): 
    model_choice, real_path, fake_path, chunk_size, sr, batch_size = args.embed_model, args.real_path, args.fake_path, args.chunk_size, args.sr, args.batch_size
    local_rank = int(os.environ.get("LOCAL_RANK", 0))
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddps = f"[{local_rank}/{world_size}]"  # string for distributed computing info, e.g. "[1/8]" 

    accelerator = Accelerator()
    hprint = HostPrinter(accelerator)  # hprint only prints on head node
    device = accelerator.device  # get_device()
    hprint(f"{ddps} args = {args}")
    hprint(f'{ddps} Using device: {device}')
    
 
    """
    # get the list(s) of audio files
    real_filenames = get_audio_filenames(real_path)
    #hprint(f"{ddps} real_path, real_filenames = {real_path}, {real_filenames}")
    fake_filenames = get_audio_filenames(fake_path)
    minlen = len(real_filenames)
    if len(real_filenames) != len(fake_filenames):
        hprint(f"{ddps} WARNING: len(real_filenames)=={len(real_filenames)} != len(fake_filenames)=={len(fake_filenames)}. Truncating to shorter list") 
        minlen = min( len(real_filenames) , len(fake_filenames) )
    
    # subdivide file lists by process number
    num_per_proc = minlen // world_size
    start = local_rank * num_per_proc
    end =  minlen if local_rank == world_size-1 else (local_rank+1) * num_per_proc
    #print(f"{ddps} start, end = ",start,end) 
    real_filenames, fake_filenames = real_filenames[start:end], fake_filenames[start:end]
    """

    # setup embedder and dataloader
    embedder = setup_embedder(model_choice, device)
    hprint(f"{ddps} Embedder '{model_choice}' ready to go!")

    real_dataset = AudioDataset(real_path, augs='Stereo(), PhaseFlipper()', sample_size=chunk_size, return_dict=True)
    fake_dataset = AudioDataset(fake_path, augs='Stereo(), PhaseFlipper()', sample_size=chunk_size, return_dict=True)
    real_dl = DataLoader(real_dataset, batch_size=16, shuffle=False)
    fake_dl = DataLoader(fake_dataset, batch_size=16, shuffle=False)
    
    real_dl, fake_dl, embedder = accelerator.prepare( real_dl, fake_dl, embedder )  # prepare handles distributing things among GPUs
    
    # note that we don't actually care if real & fake files are pulled in the same order; we'll be comparing the *distributions* of the data.
    for dl, name in zip([real_dl, fake_dl],['real','fake']):
        newdir_already = False
        for i, data_dict in enumerate(dl):
            audio, filename_batch = data_dict['inputs'], data_dict['filename']
            if not newdir_already: 
                p = Path( filename_batch[0] )
                dir_already = True
                newdir = str(p.parents[0])+"_emb"
                hprint(f"newdir = {newdir}")
                makedir(newdir) 
                
            #print(f"{ddps} i = {i}/{len(real_dataset)}, filename = {filename_batch[0]}")
            audio = audio.to(device)
            while len(audio.shape) < 3: 
                audio = audio.unsqueeze(0) # add batch and/or channel dims 
                
            embeddings = embedder.get_audio_embedding_from_data(audio.mean(dim=1).to(device), use_tensor=True).to(audio.dtype)
            hprint(f"embeddings.shape = {embeddings.shape}")
            # TODO: for now we'll just dump each batch on each proc to its own file; this could be improved
            outfilename = f"{newdir}/emb_p{local_rank}_b{i}.pt"
            print(f"{ddps} Saving embeddings to {outfilename}")
            torch.save(embeddings, outfilename)
            

## Calling sequence
```
accelerate launch fad_pytorch/fad_embed.py clap real/ fake/
```

In [ ]:
#| export
def main(): 
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('embed_model', help='chioce of embedding model', default='clap')
    parser.add_argument('real_path', help='Path of files of real audio', default='real/')
    parser.add_argument('fake_path', help='Path of files of fake audio', default='fake/')
    parser.add_argument('--chunk_size', type=int, default=24000, help='Length of chunks (in audio samples) to embed')
    parser.add_argument('--batch_size', type=int, default=16, help='Batch size for computing embeddings')
    parser.add_argument('--sr', type=int, default=48000, help='sample rate (will resample inputs at this rate)')
    args = parser.parse_args()
    embed_all(args)

In [ ]:
#| export
if __name__ == '__main__' and "get_ipython" not in dir():
    main()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()